In [1]:
import os
import sqlite3

import matplotlib.pyplot as plt
import numpy as np
import nfts.dataset

import pandas as pd
import math

nfts_merged = pd.read_csv("data/nfts_merged.csv")

In [3]:
dirname = os.path.dirname(os.path.abspath("__file__"))
DATASET_PATH = os.path.join(dirname, "data/nfts.sqlite")
ds = nfts.dataset.FromSQLite(DATASET_PATH)

nfts_df = ds.load_dataframe("nfts")
nfts_df.rename(columns={'address':"nft_address"}, inplace=True)
nfts_df.head()

,nft_address,name,symbol
0,0xF4ee95274741437636e748DdAc70818B4ED7d043,The Doge Pound,DOGGY
1,0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623,BoredApeKennelClub,BAKC
2,0x7403AC30DE7309a0bF019cdA8EeC034a5507cbB3,Characters,CHAR
3,0x511372B44231a31527025a3D273C1dc0a83D77aF,Maps Restored,MAP
4,0x2E956Ed3D7337F4Ed4316A6e8F2EdF74BF84bb54,Neon Junkies,JUNKIES


Collections with most contracts

In [4]:
collections_df = nfts_df.groupby('name', as_index=False).size().rename(columns={'size':'num_addresses'})
collections_df.sort_values('num_addresses', ascending=False, inplace=True)

collections_df.head()

,name,num_addresses
4894,NFT Basket,68
0,,19
4933,NFT-LOTTERY,16
7406,Tinlake Loan Token,13
568,Artiikorea,9


# Datasets Merging

## Tables loading and projection

In [5]:
mints_df = ds.load_dataframe("mints")
mints_df = mints_df[['nft_address', 'transaction_value', 'timestamp']]

In [ ]:
transfers_df = ds.load_dataframe("transfers")
transfers_df = transfers_df[['nft_address', 'transaction_value']]

transfers_stats_df = ds.load_dataframe("transfer_statistics_by_address")
transfers_stats_df.rename(columns={'address':"owner_address"}, inplace=True)

In [ ]:
current_owners_df = ds.load_dataframe("current_owners")
current_owners_df = current_owners_df[['nft_address', 'owner']]

In [ ]:
current_market_values_df = ds.load_dataframe("current_market_values")
current_market_values_df = current_market_values_df[['nft_address', 'market_value']]

In [ ]:

# TODO: remove cell

print(list(nfts_df.keys()), "nfts")

print(list(mints_df.keys()), "mints")
print(list(transfers_df.keys()), "transfers")
print(list(transfers_stats_df.keys()), "transfers stats")
print(list(current_owners_df.keys()), "owners")
print(list(current_market_values_df.keys()), "values")

['nft_address', 'name', 'symbol'] nfts
['nft_address', 'transaction_value', 'timestamp'] mints
['nft_address', 'transaction_value'] transfers
['owner_address', 'transfers_out', 'transfers_in'] transfers stats
['nft_address', 'owner'] owners
['nft_address', 'market_value'] values


There's a total of 9388 NFT unique addresses, as in the *nfts dataset*

None of the other addresses contains this number of addresses, therefore there are addresses withoutdata on minting, owners and value

In [ ]:

print("nft unique addresses:\t", nfts_df["nft_address"].nunique(), " / ", len(nfts_df))
print("Mints unique addresses:\t", mints_df["nft_address"].nunique(), " / ", len(mints_df))
print("Transfers unique addresses:\t", transfers_df["nft_address"].nunique(), " / ", len(transfers_df))
print("Owners unique addresses:\t", current_owners_df["nft_address"].nunique(), " / ", len(current_owners_df))
print("Values unique addresses:\t", current_market_values_df["nft_address"].nunique(), " / ", len(current_market_values_df))


nft unique addresses:	 9388  /  9388
Mint unique addresses:	 7647  /  6667282
Transfers unique addresses:	 6701  /  4514729
Owners unique addresses:	 9292  /  7020950
Values unique addresses:	 9292  /  7020950


## GroupBy and Aggregate

In [ ]:

# TODO: for each nft_address, number of transfers (see transfers.ipynb)

nfts_merged_df = nfts_df

# for mints, take the average transaction_value and the difference in timestamp
mints_agg_df = mints_df.groupby("nft_address", as_index=False).aggregate({'transaction_value': 'mean', 
                                                                            'timestamp': lambda x: x.max() - x.min()})
mints_agg_df.rename(columns={"transaction_value":"mints_avg_transaction_value", "timestamp":"mints_timestamp_range"}, inplace=True)
nfts_merged_df = nfts_merged_df.merge(mints_agg_df, on="nft_address", how="left")

# for transfers, take the average transaction_value
transfers_agg_df = transfers_df.groupby("nft_address", as_index=False).aggregate(transfers_avg_transaction_value=pd.NamedAgg(column="transaction_value", aggfunc="mean"),
                                                                                    transfers_count=pd.NamedAgg(column="transaction_value", aggfunc="count"))
nfts_merged_df = nfts_merged_df.merge(transfers_agg_df, on="nft_address", how="left")

# number of current owners
owners_agg_df = current_owners_df.groupby("nft_address", as_index=False).count()
owners_agg_df.rename(columns={"owner":"num_owners"}, inplace=True) 
nfts_merged_df = nfts_merged_df.merge(owners_agg_df, on="nft_address", how="left")

# average market values
values_agg_df = current_market_values_df.groupby("nft_address", as_index=False).mean()
values_agg_df.rename(columns={"market_value":"avg_market_value"}, inplace=True) 
nfts_merged_df = nfts_merged_df.merge(values_agg_df, on="nft_address", how="left")

Costs conversion to ETH

In [ ]:
nfts_merged_df["mints_avg_transaction_value"] = nfts_merged_df["mints_avg_transaction_value"] / (10**18)
nfts_merged_df["transfers_avg_transaction_value"] = nfts_merged_df["transfers_avg_transaction_value"] / (10**18)
nfts_merged_df["avg_market_value"] = nfts_merged_df["avg_market_value"] / (10**18)

nfts_merged_df.to_csv("data/nfts_merged.csv", index=False)
nfts_merged_df.head()

,nft_address,name,symbol,mints_avg_transaction_value,mints_timestamp_range,transfers_avg_transaction_value,transfers_count,num_owners,avg_market_value
0,0xF4ee95274741437636e748DdAc70818B4ED7d043,The Doge Pound,DOGGY,0.833024,444184.0,0.809497,15352.0,10000.0,1.013126
1,0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623,BoredApeKennelClub,BAKC,0.000000,605983.0,1.782388,18137.0,9602.0,1.701389
2,0x7403AC30DE7309a0bF019cdA8EeC034a5507cbB3,Characters,CHAR,0.019601,1508463.0,0.144810,4308.0,7242.0,0.077697
3,0x511372B44231a31527025a3D273C1dc0a83D77aF,Maps Restored,MAP,0.000000,1802082.0,0.107217,791.0,4440.0,0.017773
4,0x2E956Ed3D7337F4Ed4316A6e8F2EdF74BF84bb54,Neon Junkies,JUNKIES,0.219857,1935393.0,0.088245,3152.0,9095.0,0.163138


# Principal Component Analysis

- *after attribute selection*
- execute PCA (look for a library)
- evaluate variance and advantages
- choose if using it or nah (and in case, with how many dimensions)

# Clustering

## Clustering Tendency

- Hopkins
- VAT

## Clustering Quality

- Choice of *K*
    - Elbow Point
    - distance from random *(?)*
- Silhouette (inter/intra measures)

# Conclusions